In [1]:
from call_nlp_api import *
from google.cloud import bigquery
import pandas_gbq
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter("always", category=PendingDeprecationWarning) 
warnings.simplefilter("always", category=DeprecationWarning) 

In [3]:
bq_client = bigquery.Client(location="US", project="ut-goog")
query = """
  SELECT * FROM `nl_api.Karan_sparky_test`
  """
dataset = "nl_api"
table = "nl_api_data"

# Load queries into dataframe and load review checkpoints.
df = load_query_results_to_df(bq_client, query)

Query results loaded into dataframe!


In [4]:
df.head()

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,...,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,sentiment_score,sentiment_magnitude,entities,entity_types,entity_sentiment_scores,entity_sentiment_magnitudes
complaint_id,,,,,,,,,,,,,,,,,,,,,
2090321,2016-08-31,Student loan,Non-federal student loan,Getting a loan,Can't qualify for a loan,I attended XXXX XXXX from XXXX XXXX XXXX. I wa...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",TN,37923,...,2016-08-31,Closed with explanation,True,False,-0.1,12.1,"['XXXX XXXX', 'school', 'interest rates', 'XXX...","['OTHER', 'ORGANIZATION', 'OTHER', 'CONSUMER_G...","[-0.10000000149011612, 0, 0.10000000149011612,...","[1.100000023841858, 0, 0.5, 0.8999999761581421..."
3411046,2019-10-19,Checking or savings account,Savings account,Managing an account,Deposits and withdrawals,"On XX/XX/2019 at XXXX XXXX, I received a call ...",Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",NY,112XX,...,2019-10-19,Closed with non-monetary relief,True,None,-0.5,23.1,"['representative', 'XXXXXXXX XXXX', 'account',...","['PERSON', 'OTHER', 'OTHER', 'ORGANIZATION', '...","[0, -0.20000000298023224, -0.20000000298023224...","[0.4000000059604645, 3.799999952316284, 1.0, 2..."
2828978,2018-02-28,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,"On XX/XX/2018, I noted two checking/saving acc...",Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",CA,928XX,...,2018-02-28,Closed with non-monetary relief,True,None,-0.5,7.5,"['XX/XX', 'accounts', 'account', 'credit card ...","['OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', ...","[-0.10000000149011612, 0, -0.800000011920929, ...","[1.899999976158142, 0.10000000149011612, 0.800..."
3328539,2019-08-04,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,I have a complaint against Bank of America for...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",GA,None,...,2019-08-04,Closed with explanation,True,None,-0.5,7.8,"['Bank of America', 'check', 'calls', 'staff',...","['ORGANIZATION', 'OTHER', 'OTHER', 'PERSON', '...","[-0.30000001192092896, -0.10000000149011612, -...","[8.800000190734863, 1.2000000476837158, 0.3000..."
2962956,2018-07-14,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,XX/XX/XXXX I received an email about a previou...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",FL,333XX,...,2018-07-14,Closed with explanation,True,None,-0.4,1.2,"['XX/XX', 'paperwork', 'claim', 'email', 'frau...","['OTHER', 'OTHER', 'OTHER', 'OTHER', 'EVENT', ...","[-0.10000000149011612, -0.10000000149011612, -...","[0.20000000298023224, 0.699999988079071, 0.400..."


In [10]:
df['product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    147298
Debt collection                                                                 107741
Mortgage                                                                         61900
Credit card or prepaid card                                                      32643
Credit reporting                                                                 31588
Student loan                                                                     25186
Checking or savings account                                                      19418
Credit card                                                                      18838
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             8283
Money transfer, virtual currency, or money 

In [11]:
def prod(x):
    if(x=='Credit card'):
        return 'Credit card or prepaid card'
    elif(x=='Prepaid card'):
        return 'Credit card or prepaid card'
    elif(x=='Payday loan'):
        return 'Payday loan, title loan, or personal loan'
    elif(x=='Money transfers'):
        return 'Money transfer, virtual currency, or money service'
    elif(x=='Virtual currency'):
        return 'Money transfer, virtual currency, or money service'
    else:
        return x

df['product'] = df['product'].apply(prod)

In [12]:
df['product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    147298
Debt collection                                                                 107741
Mortgage                                                                         61900
Credit card or prepaid card                                                      52931
Credit reporting                                                                 31588
Student loan                                                                     25186
Checking or savings account                                                      19418
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                                9476
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             8283
Payday loan, title loan, or personal loan  

In [44]:
temp = pd.DataFrame((df.groupby([df['subproduct']])['issue'].count()/len(df)).sort_values(ascending=False)).reset_index()

In [46]:
temp['issue'] = temp['issue']*100

In [54]:
temp[temp['issue']>1]

,subproduct,issue
0,Credit reporting,28.990896
1,Checking account,5.287507
2,General-purpose credit card or charge card,5.005858
3,I do not know,4.236408
4,Other debt,3.875438
5,Conventional home mortgage,3.533393
6,Credit card debt,3.017205
7,Conventional fixed mortgage,2.931643
8,Federal student loan servicing,2.909095
9,"Other (i.e. phone, health club, etc.)",2.494172


In [52]:
mask = list(temp[temp['issue']<1]['subproduct'])

In [58]:
def subprod(x):
    if(x in mask):
        return 'Other'
    elif(x=='Other (i.e. phone, health club, etc.)'):
        return 'Other'
    elif(x=='Loan'):
        return 'Other loan'
    else:
        return x
    
df['subproduct'] = df['subproduct'].apply(subprod)

In [59]:
df['subproduct'].value_counts()

Credit reporting                              144003
Other                                          70103
Checking account                               26264
General-purpose credit card or charge card     24865
I do not know                                  21043
Other debt                                     19250
Conventional home mortgage                     17551
Credit card debt                               14987
Conventional fixed mortgage                    14562
Federal student loan servicing                 14450
Medical debt                                   11558
FHA mortgage                                   11516
Credit card                                     7520
Medical                                         6990
Other loan                                      6789
Non-federal student loan                        6437
Store credit card                               5809
Installment loan                                5750
Vehicle loan                                  

In [64]:
temp = pd.DataFrame((df.groupby([df['issue']])['product'].count()/len(df)).sort_values(ascending=False)).reset_index()

In [65]:
temp[temp['product']>0.01]

,issue,product
0,Incorrect information on your report,0.163622
1,Problem with a credit reporting company's inve...,0.080462
2,Attempts to collect debt not owed,0.059513
3,Incorrect information on credit report,0.042714
4,Improper use of your report,0.041545
5,Cont'd attempts collect debt not owed,0.035098
6,"Loan servicing, payments, escrow account",0.029639
7,Communication tactics,0.028032
8,Trouble during payment process,0.027507
9,Written notification about debt,0.025564


In [66]:
mask = list(temp[temp['product']<0.01]['issue'])

In [67]:
def issue(x):
    if(x in mask):
        return 'Other'
    elif(x=='Dealing with your lender or servicer'):
        return 'Dealing with lender or servicer'
    elif(x=='Dealing with my lender or servicer'):
        return 'Dealing with lender or servicer'
    else:
        return x
    
df['issue'] = df['issue'].apply(issue)

In [68]:
df['issue'].value_counts()

Other                                                                               131360
Incorrect information on your report                                                 81274
Problem with a credit reporting company's investigation into an existing problem     39967
Attempts to collect debt not owed                                                    29561
Incorrect information on credit report                                               21217
Improper use of your report                                                          20636
Cont'd attempts collect debt not owed                                                17434
Dealing with lender or servicer                                                      17252
Loan servicing, payments, escrow account                                             14722
Communication tactics                                                                13924
Trouble during payment process                                                       13663

In [70]:
temp = pd.DataFrame((df.groupby([df['subissue']])['product'].count()/len(df)).sort_values(ascending=False)).reset_index()
temp[temp['product']>0.01]

,subissue,product
0,Information belongs to someone else,0.069124
1,Their investigation did not fix an error on yo...,0.057596
2,Account status incorrect,0.032304
3,Account information incorrect,0.031263
4,Debt is not yours,0.030478
5,Credit inquiries on your report that you don't...,0.027013
6,Debt was paid,0.022699
7,Debt is not mine,0.020344
8,Attempted to collect wrong amount,0.016744
9,Didn't receive enough information to verify debt,0.015747


In [71]:
mask = list(temp[temp['product']<0.01]['subissue'])

In [72]:
def subissue(x):
    if(x in mask):
        return 'Other'
    elif(x=='Debt is not yours'):
        return 'Incorrect debt'
    elif(x=='Debt is not mine'):
        return 'Incorrect debt'
    elif(x=='Account status'):
        return 'Account status incorrect'
    else:
        return x
    
df['subissue'] = df['subissue'].apply(subissue)

In [73]:
df['subissue'].value_counts()

Other                                                       174525
Information belongs to someone else                          34335
Their investigation did not fix an error on your report      28609
Incorrect debt                                               25244
Account status incorrect                                     21185
Account information incorrect                                15529
Credit inquiries on your report that you don't recognize     13418
Debt was paid                                                11275
Attempted to collect wrong amount                             8317
Didn't receive enough information to verify debt              7822
Frequent or repeated calls                                    7639
Information is not mine                                       7102
Reporting company used your report improperly                 6737
Debt was result of identity theft                             6314
Name: subissue, dtype: int64

In [74]:
df

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,...,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,sentiment_score,sentiment_magnitude,entities,entity_types,entity_sentiment_scores,entity_sentiment_magnitudes
complaint_id,,,,,,,,,,,,,,,,,,,,,
2090321,2016-08-31,Student loan,Non-federal student loan,Other,Other,I attended XXXX XXXX from XXXX XXXX XXXX. I wa...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",TN,37923,...,2016-08-31,Closed with explanation,True,False,-0.1,12.1,"['XXXX XXXX', 'school', 'interest rates', 'XXX...","['OTHER', 'ORGANIZATION', 'OTHER', 'CONSUMER_G...","[-0.10000000149011612, 0, 0.10000000149011612,...","[1.100000023841858, 0, 0.5, 0.8999999761581421..."
3411046,2019-10-19,Checking or savings account,Other,Managing an account,Other,"On XX/XX/2019 at XXXX XXXX, I received a call ...",Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",NY,112XX,...,2019-10-19,Closed with non-monetary relief,True,None,-0.5,23.1,"['representative', 'XXXXXXXX XXXX', 'account',...","['PERSON', 'OTHER', 'OTHER', 'ORGANIZATION', '...","[0, -0.20000000298023224, -0.20000000298023224...","[0.4000000059604645, 3.799999952316284, 1.0, 2..."
2828978,2018-02-28,Checking or savings account,Checking account,Managing an account,Other,"On XX/XX/2018, I noted two checking/saving acc...",Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",CA,928XX,...,2018-02-28,Closed with non-monetary relief,True,None,-0.5,7.5,"['XX/XX', 'accounts', 'account', 'credit card ...","['OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', ...","[-0.10000000149011612, 0, -0.800000011920929, ...","[1.899999976158142, 0.10000000149011612, 0.800..."
3328539,2019-08-04,Checking or savings account,Checking account,Managing an account,Other,I have a complaint against Bank of America for...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",GA,None,...,2019-08-04,Closed with explanation,True,None,-0.5,7.8,"['Bank of America', 'check', 'calls', 'staff',...","['ORGANIZATION', 'OTHER', 'OTHER', 'PERSON', '...","[-0.30000001192092896, -0.10000000149011612, -...","[8.800000190734863, 1.2000000476837158, 0.3000..."
2962956,2018-07-14,Checking or savings account,Checking account,Managing an account,Other,XX/XX/XXXX I received an email about a previou...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",FL,333XX,...,2018-07-14,Closed with explanation,True,None,-0.4,1.2,"['XX/XX', 'paperwork', 'claim', 'email', 'frau...","['OTHER', 'OTHER', 'OTHER', 'OTHER', 'EVENT', ...","[-0.10000000149011612, -0.10000000149011612, -...","[0.20000000298023224, 0.699999988079071, 0.400..."


In [75]:
table_name = "Karan_rolled_up_data"
dataset = "nl_api"
df.index = df.index.astype(int)
dataset = bq_client.dataset(dataset)
table_ref = dataset.table(table_name)

job_config = bigquery.LoadJobConfig()
job_config.write_disposition = "WRITE_TRUNCATE"
job = bq_client.load_table_from_dataframe(df, table_ref, location="US",
                                            job_config=job_config)
job.result()
assert job.state == "DONE"
print("BQ table updated:", table_name)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  after removing the cwd from sys.path.


BQ table updated: Karan_rolled_up_data
